In [ ]:
# Load Data
import numpy as np
import scipy.io as sio
datas = sio.loadmat('dataset_mapping.mat')
bed_select=datas['bed_select']
bed_whole=datas['bed_whole']
pillow_up=datas['pillow_up']
person = datas['person']
posture = datas['posture']

In [ ]:
i=15
ood_idx = np.where(posture==i)[0]
indis_idx = np.setdiff1d(np.arange(len(posture)), ood_idx)
indis_input = bed_select[indis_idx,:,:]
ood_input = bed_select[ood_idx,:,:]
indis_output = pillow_up[indis_idx,:,:]
ood_output = pillow_up[ood_idx,:,:]
X_tr, X_te, y_tr, y_te = train_test_split(indis_input, indis_output,train_size=0.9)
scaler = MinMaxScaler()
scaler2 = MinMaxScaler()
reshaped_xtr = X_tr.reshape(X_tr.shape[0],-1)
reshaped_xte = X_te.reshape(X_te.shape[0],-1)
reshaped_ytr = y_tr.reshape(y_tr.shape[0],-1)
reshaped_yte = y_te.reshape(y_te.shape[0],-1)
reshaped_xood = ood_input.reshape(ood_input.shape[0],-1)
reshaped_yood = ood_output.reshape(ood_output.shape[0],-1)

normalized_xtr1 = scaler.fit_transform(reshaped_xtr)
normalized_xte1 = scaler.transform(reshaped_xte)
normalized_xood1 = scaler.transform(reshaped_xood)
normalized_ytr1 = scaler2.fit_transform(reshaped_ytr)
normalized_yte1 = scaler2.transform(reshaped_yte)
normalized_yood1 = scaler2.transform(reshaped_yood)

normalized_xtr = normalized_xtr1.reshape(X_tr.shape)
normalized_xte = normalized_xte1.reshape(X_te.shape)
normalized_ytr = normalized_ytr1.reshape(y_tr.shape)
normalized_yte = normalized_yte1.reshape(y_te.shape)
normalized_xood = normalized_xood1.reshape(ood_input.shape)
normalized_yood = normalized_yood1.reshape(ood_output.shape)

input_shape = (16,40,1)
output_shape = (32,48,1)
normalized_xtr = normalized_xtr[:,:,:,np.newaxis]
normalized_xte = normalized_xte[:,:,:,np.newaxis]
normalized_ytr = normalized_ytr[:,:,:,np.newaxis]
normalized_yte = normalized_yte[:,:,:,np.newaxis]
normalized_xood = normalized_xood[:,:,:,np.newaxis]
normalized_yood = normalized_yood[:,:,:,np.newaxis]

In [ ]:
# Create Model
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from matplotlib import pyplot
def create_model(input_shape=(16,40,1),output_shape=(32,48,1)):
    init = RandomNormal(stddev=0.01)
    in_image = Input(shape=input_shape)
    n_filters1=32
    g1 = Conv2D(n_filters1, (4,6), strides=(1,1), kernel_initializer=init)(in_image)
    g1 = LeakyReLU(alpha=0.2)(g1)
    g2= Conv2D(n_filters1*2, (3,5), strides=(1,1), kernel_initializer=init)(g1)
    g2 = LeakyReLU(alpha=0.2)(g2)
    g3= Conv2D(n_filters1*2, (3,5), strides=(1,1), kernel_initializer=init)(g2)
    g3 = LeakyReLU(alpha=0.2)(g3)
    g4= Conv2D(n_filters1*2, (2,4), strides=(1,1), kernel_initializer=init)(g3)
    g4 = LeakyReLU(alpha=0.2)(g4)
    g4= Conv2D(n_filters1*2, (2,4), strides=(1,1), kernel_initializer=init)(g4)
    g4 = LeakyReLU(alpha=0.2)(g4)
    g4 = Activation('relu')(g4)
    g5=Conv2DTranspose(n_filters1*2, (2,4), strides=(1,1), kernel_initializer=init)(g4)
    g5 = LeakyReLU(alpha=0.2)(g5)
    g5=Conv2DTranspose(n_filters1*2, (7,7), strides=(1,1), kernel_initializer=init)(g5)
    g5 = LeakyReLU(alpha=0.2)(g5)
    g6=Conv2DTranspose(n_filters1*2, (7,7), strides=(1,1), kernel_initializer=init)(g5)
    g6 = LeakyReLU(alpha=0.2)(g6)
    g7=Conv2DTranspose(n_filters1, (7,7), strides=(1,1), kernel_initializer=init)(g6)
    g7 = LeakyReLU(alpha=0.2)(g7)
    g8=Conv2DTranspose(1, (7,7), strides=(1,1), kernel_initializer=init)(g7)
    g8 = LeakyReLU(alpha=0.2)(g8)
    out_image = Activation('relu')(g8)
    model = Model(in_image, out_image)
    return model

In [ ]:
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras import losses
normalized_xtr = np.squeeze(normalized_xtr)
normalized_xte = np.squeeze(normalized_xte)
normalized_ytr = np.squeeze(normalized_ytr)
normalized_yte = np.squeeze(normalized_yte)
normalized_xood = np.squeeze(normalized_xood)
normalized_yood = np.squeeze(normalized_yood)
y_te = np.squeeze(y_te)
ood_output = np.squeeze(ood_output)
sim_mat = np.zeros((normalized_xtr.shape[0],))
for j in range(normalized_xtr.shape[0]):
    tmp_mat = np.zeros((normalized_xood.shape[0],))
    for k in range(normalized_xood.shape[0]):
        s = ssim(normalized_xtr[j,:,:],normalized_xood[k,:,:], data_range=1)
        tmp_mat[k] = s
    sim_mat[j] = np.median(tmp_mat)
input_shape = (16,40,1)
output_shape = (32,48,1)
normalized_xtr = normalized_xtr[:,:,:,np.newaxis]
normalized_xte = normalized_xte[:,:,:,np.newaxis]
normalized_ytr = normalized_ytr[:,:,:,np.newaxis]
normalized_yte = normalized_yte[:,:,:,np.newaxis]
normalized_xood = normalized_xood[:,:,:,np.newaxis]
normalized_yood = normalized_yood[:,:,:,np.newaxis]
model = create_model(input_shape, output_shape)
lr = 0.1
batch_size = 32
train_epoch = 128
opt = tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(loss='mean_absolute_error', optimizer='adam')
modelname = 'best_model_weight_' + str(i) + '.h5'
checkpoint = ModelCheckpoint(modelname, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
sample_weights = sim_mat[:,np.newaxis]
history = model.fit(normalized_xtr,normalized_ytr,sample_weight = sample_weights,batch_size=batch_size, epochs=150,validation_data=(normalized_xte,normalized_yte), callbacks=[checkpoint])
import keras
model = keras.models.load_model(modelname)
y_pred_norm=model.predict(normalized_xte)
reshaped_ypred = y_pred_norm.reshape(y_pred_norm.shape[0],-1)
y_pred1 = scaler2.inverse_transform(reshaped_ypred)
y_pred = y_pred1.reshape(y_pred_norm.shape)
y_te = y_te[:,:,:,np.newaxis]
print(np.mean(np.abs(y_pred-y_te)))

ood_output = ood_output[:,:,:,np.newaxis]
y_pred_norm_ood=model.predict(normalized_xood)
reshaped_ypred_ood = y_pred_norm_ood.reshape(y_pred_norm_ood.shape[0],-1)
y_pred_ood1 = scaler2.inverse_transform(reshaped_ypred_ood)
y_pred_ood = y_pred_ood1.reshape(ood_output.shape)

print(np.mean(np.abs(y_pred_ood-ood_output)))

In [ ]:
a = y_pred[0,:,:,0]
b = y_te[0,:,:,0]

In [ ]:
import matplotlib.pyplot as plt

# Create a sample matrix
matrix = a

# Plot the colormap of the matrix
plt.imshow(matrix, cmap='viridis')
#plt.colorbar()

# Show the plot
plt.show()
matrix = b

# Plot the colormap of the matrix
plt.imshow(matrix, cmap='viridis')
#plt.colorbar()

# Show the plot
plt.show()

In [ ]:
c= y_pred_ood[3,:,:,0]
d = ood_output[3,:,:,0]

In [ ]:
matrix = c

# Plot the colormap of the matrix
plt.imshow(matrix, cmap='viridis')
#plt.colorbar()

# Show the plot
plt.show()
matrix = d

# Plot the colormap of the matrix
plt.imshow(matrix, cmap='viridis')
#plt.colorbar()

# Show the plot
plt.show()

In [ ]:
y_max = np.max(y_te.reshape(len(y_te),-1),axis = 1)
percent_err = np.mean(np.abs(y_pred-y_te)/y_max[:,np.newaxis,np.newaxis,np.newaxis])

In [ ]:
percent_err

In [ ]:
y_max = np.max(ood_output.reshape(len(ood_output),-1),axis = 1)
percent_err = np.mean(np.abs(y_pred_ood-ood_output)/y_max[:,np.newaxis,np.newaxis,np.newaxis])

In [ ]:
percent_err